In [28]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

In [29]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.7
)

router_llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [30]:
MODEL_CONFIG = {
    "technical": "You are a senior software engineer. Provide precise, technical, code-focused debugging solutions with examples.",
    "billing": "You are a polite and empathetic billing support agent. Handle refunds, payments, subscriptions, and policy-related concerns clearly.",
    "general": "You are a friendly and helpful assistant for general conversations and casual questions."
}

In [31]:
def route_prompt(user_input: str) -> str:
    prompt = f"""
    Classify the following user query into exactly one of these categories:
    [technical, billing, general]

    Return ONLY the category name.

    Query: {user_input}
    """

    response = router_llm.invoke(prompt)
    return response.content.strip().lower()

In [32]:
def process_request(user_input: str) -> str:
    category = route_prompt(user_input)

    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]

    messages = [
        ("system", system_prompt),
        ("human", user_input)
    ]

    response = llm.invoke(messages)

    return f"[ROUTED → {category.upper()} EXPERT]\n\n{response.content}"

In [33]:
test_queries = [
    "My python script is throwing IndexError on line 5",
    "I was charged twice for my subscription this month",
    "Tell me a joke"
]

for q in test_queries:
    print("User:", q)
    print(process_request(q))
    print("-"*80)

User: My python script is throwing IndexError on line 5
[ROUTED → TECHNICAL EXPERT]

To provide a precise debugging solution, I'll need to see the code. However, I can give you a general idea of how to debug an `IndexError` in Python.

An `IndexError` typically occurs when you're trying to access an element in a list or other sequence that doesn't exist. For example, if you have a list of 3 elements, the valid indices are 0, 1, and 2. If you try to access the element at index 3, you'll get an `IndexError`.

Here's an example of code that would throw an `IndexError`:
```python
my_list = [1, 2, 3]
print(my_list[3])  # IndexError: list index out of range
```
To debug this, you would need to check why you're trying to access an index that doesn't exist. Here are some possible solutions:

1. **Check the length of the list**: Make sure the list has enough elements to access the index you're trying to access.
```python
my_list = [1, 2, 3]
if len(my_list) > 3:
    print(my_list[3])
else:
    p

In [34]:
def fetch_bitcoin_price():
    return "Current Bitcoin Price: $52,300 (Mock Data)"

In [35]:
def route_prompt(user_input: str) -> str:
    if "bitcoin" in user_input.lower():
        return "tool"

    prompt = f"""
    Classify the following user query into exactly one of these categories:
    [technical, billing, general]

    Return ONLY the category name.

    Query: {user_input}
    """

    response = router_llm.invoke(prompt)
    return response.content.strip().lower()

In [36]:
def process_request(user_input: str) -> str:
    category = route_prompt(user_input)

    if category == "tool":
        return "[ROUTED → TOOL EXPERT]\n\n" + fetch_bitcoin_price()

    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]

    messages = [
        ("system", system_prompt),
        ("human", user_input)
    ]

    response = llm.invoke(messages)

    return f"[ROUTED → {category.upper()} EXPERT]\n\n{response.content}"

In [37]:
print(process_request("What is the current price of Bitcoin?"))

[ROUTED → TOOL EXPERT]

Current Bitcoin Price: $52,300 (Mock Data)
